In [1]:
from PIL import Image
import pandas as pd
import numpy as np
import cv2 as cv
from glob import glob
from sklearn.model_selection import train_test_split
from mitosplit_net import util
from training_functions import create_model,train_model
from albumentations import Compose, Rotate, RandomRotate90, HorizontalFlip, Flip, VerticalFlip
from tqdm import tqdm
import os
from os import path
import random as r
from import_augmentation_function import import_fun, aug_fun, import_aug_fun, normalization_fun, normalization_fun_one
import matplotlib.pyplot as plt
import tensorflow as tf 
import imageio


In [8]:
files_dir = r'C:\Users\roumba\Documents\Software\deep-events\training_data'
images_dir = '220922_mtstaygold_U2OS_iSIM_fl_pos'

In [7]:
import multiprocessing
import test_multiprocess as tm
sigmas = [1,2,7,8,9]
number_of_augmentations = 40
for sigma in sigmas:
    print(sigma)
    p = multiprocessing.Process(target=tm.load_aug_train, args=(files_dir, images_dir, sigma, number_of_augmentations)) 
    p.start() 
    p.join()


3
4
5
